### import packages

In [59]:
#import required packages
import pandas as pd
import gensim
# NLTK Stop words
import pyLDAvis
from nltk.corpus import stopwords
import pickle
stop_words = stopwords.words('english')
from gensim import corpora
# Import Dataset
from gensim.utils import simple_preprocess
import pyLDAvis
import pyLDAvis.gensim
import spacy
import nltk
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn import linear_model

In [10]:
# loads pipeline package for Language model (English)
nlp = spacy.load("en_core_web_sm")

In [11]:
#seting the maximum column width to display all columns
pd.set_option('display.max_colwidth',-1)

C:\Users\manasa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


### read json file

In [12]:
#loads json file
jsn=json.loads(open('data/case_data.json').read())

In [13]:
 #converts json to dataframe
data=pd.DataFrame.from_dict(pd.json_normalize(jsn,'data'))

In [14]:
#display first 4 rows
data.head(4)

,id,name,description,metrics,industries,created_at,updated_at,function
0,493,Retargeting,Retarget customers who have already expressed interest in your products or services,Cash flow\nReturn on marketing investment (ROMI),Broadcasting\nFilm Distribution\nMedia and Entertainment\nEnterprise Software\nFinTech\nBanking\nFinancial Services\nArt\nHome Decor\nFashion\nE-Commerce\nRecruiting\nDelivery\nFood and Beverage\nE-Commerce\nFood Processing\nLifestyle\nDietary Supplements\nE-Commerce\nAI Tools\nSupply Chain Management\nMarketplace\nAnalytics\nB2B\nImage Recognition\nDigital Media\nPhotography\nPredictive Analytics\nSEO\nAdvertising\nDirect Marketing\nEmail Marketing\nE-Commerce\nAnalytics\nEvents\nTicketing\nInternet\nTravel\nFashion\nShopping\nE-Commerce\nLifestyle\nFurniture\nFashion\nE-Commerce\nFashion\nE-Commerce\nFitness\nCooking\nHospitality\nMobile\nPersonalization\nFashion\nE-Commerce\nJewelry\nLifestyle\nCustomer Service\nRetail Technology\nInternet\nE-Commerce\nFashion\nManufacturing\nInternet\nE-Commerce\nRetail\nTelecommunications\nRetail\nMobile Devices\nCustomer Service\nLifestyle\nFashion\nRetail Technology\nSoftware Solutions\nE-Commerce\nRetail\nRetail\nOpen Source\nSoftware Solutions\nHardware\nDIY\n3D Printing\nReal Estate\nClassifieds\nE-Commerce\nNews,2020-02-03T15:08:21.159Z,2020-02-03T15:08:21.159Z,Marketing
1,494,Recommendation engine,"Also called recommendation personalization system or recommendation system, these systems leverage customer data to retain and upgrade customers with personalized recommendations via email, site search or other channels",Cash flow\nReturn on marketing investment (ROMI)\nEngagement & conversion\nB2C sales,Search Engine\nRetail\nShopping\nE-Commerce\nLifestyle\nFashion\nRetail\nFashion\nWomen's\nBeauty\nCosmetics\nRetail\nE-Commerce\nFashion,2020-02-03T15:08:21.212Z,2020-02-03T15:08:21.212Z,Marketing
2,495,Social analytics & automation,"Leverage Natural Language Processing and machine vision to analyze and act upon all content generated by your actual or potential customers on social media, surveys and reviews",Customer service effectiveness\nNet Promoter Score (NPS),Cloud Computing\nSoftware Solutions\nInformation Technology\nWeb Hosting\nIaaS\nManufacturing\nAutomotive\nTransportation\nVenture Capital\nFinance\nAngel Investment\nSocial Media\nMarketing\nAdvertising\nBusiness Information Systems\nBusiness Intelligence\nInternet\nAdvertising\nTest and Measurement\nVending and Concessions,2020-02-03T15:08:21.249Z,2020-02-03T15:08:21.249Z,Marketing
3,496,Product Information Management,Manage and improve all your product information centrally to improve product discoverability and appeal,Return on marketing investment (ROMI),Communications Infrastructure\nHardware\nEnterprise Software\nVenture Capital\nFinance\nAngel Investment\nE-Commerce Platforms\nFashion\nBeauty\nRetail\nE-Commerce\nRetail Technology,2020-02-03T15:08:21.270Z,2020-02-03T15:08:21.270Z,Marketing


In [15]:
#convert series to list
data_list=list(data.loc[:,'description'].values)

In [16]:
#check the number of features
data.shape

(122, 8)

### training 'description' feature to find the target

In [17]:
#copy of data
X=data.copy()
#process description feature to find the target
X=X['description']#,'name','industries','metrics']]

### stop words  (NLTK)

In [18]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [19]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
                  'would','really','like','great','service','came','got'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod
    
def get_corpus(df):
    df['text'] = strip_newline(df)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram_mod = bigrams(words)
    bigram = [bigram_mod[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

train_corpus, train_id2word, bigram_train = get_corpus(X)


### creating a models for corpus, bigrams, bag of words

In [20]:

with open('train_corpus.pkl', 'wb') as f:
    pickle.dump(train_corpus, f)
with open('train_id2word.pkl', 'wb') as f:
    pickle.dump(train_id2word, f)
with open('bigram_train.pkl', 'wb') as f:
    pickle.dump(bigram_train, f)


In [21]:
with open('train_corpus.pkl', 'rb') as f:
    train_corpus = pickle.load(f)
with open('train_id2word.pkl', 'rb') as f:
    train_id2word = pickle.load(f)
with open('bigram_train.pkl', 'rb') as f:
    bigram_train = pickle.load(f)

### LDA Model

In [237]:
import logging
import warnings
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
num_topics=20
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True,
                           alpha=[0.01]*100,eta=[0.01]*len(train_id2word.keys()))
    lda_train.save('lda_train.model')

### traning on LDA model

In [238]:
query = 'Predict risk of churn for individual customers/clients and recommend  renegotiation strategy '
vec_bow = train_id2word.doc2bow(query.lower().split(),return_missing=True)
print(vec_bow[1].keys())


dict_keys(['and', 'churn', 'customers/clients', 'for', 'individual', 'of', 'predict', 'recommend', 'renegotiation', 'risk', 'strategy'])


In [239]:
train_vecs = []
for i in range(len(X)-1):
    top_topics = lda_train.get_document_topics(train_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    #topic_vec.extend([rev_train.iloc[i].real_counts]) 
    #topic_vec.extend([len(X.iloc[i].text)]) 
    train_vecs.append(topic_vec)

In [240]:
x = np.array(train_vecs)
y = np.array(data['function'])

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(x, y):
    # Assign CV IDX
    X_train, y_train = x[train_ind], y[train_ind]
    X_val, y_val = x[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='micro'))
    
    # Logistic Regression SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='micro'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='micro'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

C:\Users\manasa\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\manasa\anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\manasa\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\manasa\anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


Logistic Regression Val f1: 0.254 +- 0.076
Logisitic Regression SGD Val f1: 0.230 +- 0.036
SVM Huber Val f1: 0.157 +- 0.097


C:\Users\manasa\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\manasa\anaconda3\lib\site-packages\sklearn\utils\optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


### Coherence of  LDA model

In [61]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_train, texts=bigram_train, dictionary=train_id2word, coherence='c_v')
coherence_ld = coherence_model_lda.get_coherence()
coherence_ld

0.2700564599662442

### displaying 10 of the 20 topics with 15 top words for each

In [241]:
lda_train.print_topics(20,num_words=15)[:10]

[(42,
  '0.059*"improve" + 0.059*"processing" + 0.059*"analytics" + 0.059*"sales" + 0.059*"ai" + 0.059*"learning" + 0.059*"use" + 0.059*"advanced" + 0.059*"customers" + 0.059*"leverage" + 0.059*"analyze" + 0.059*"data" + 0.059*"customer" + 0.059*"systems" + 0.059*"machine"'),
 (85,
  '0.059*"data" + 0.059*"systems" + 0.059*"leverage" + 0.059*"sales" + 0.059*"customer" + 0.059*"use" + 0.059*"customers" + 0.059*"analyze" + 0.059*"learning" + 0.059*"analytics" + 0.059*"ai" + 0.059*"processing" + 0.059*"machine" + 0.059*"natural" + 0.059*"advanced"'),
 (67,
  '0.378*"customer" + 0.314*"satisfaction" + 0.303*"improve" + 0.000*"machine" + 0.000*"advanced" + 0.000*"learning" + 0.000*"leverage" + 0.000*"data" + 0.000*"analytics" + 0.000*"sales" + 0.000*"ai" + 0.000*"customers" + 0.000*"systems" + 0.000*"analyze" + 0.000*"use"'),
 (44,
  '0.059*"improve" + 0.059*"processing" + 0.059*"analytics" + 0.059*"sales" + 0.059*"ai" + 0.059*"learning" + 0.059*"use" + 0.059*"advanced" + 0.059*"customers" 

In [187]:
topics_matrix = lda_train.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

["('sales', 0.89380693)", "('customer', 0.09980079)", "('processing', 0.0004263404)", "('improve', 0.0004262475)", "('leverage', 0.00042624533)", "('systems', 0.00042624414)", "('data', 0.00042614664)", "('machine', 0.00042614664)", "('natural', 0.00042614664)", "('analytics', 0.00042614664)", "('advanced', 0.00042614664)", "('ai', 0.00042614664)", "('learning', 0.00042614664)", "('customers', 0.00042604975)", "('analyze', 0.00042604975)", "('use', 0.00042604975)", "('satisfaction', 0.00042604975)"]

["('analytics', 0.05887052)", "('customer', 0.05885487)", "('systems', 0.058850233)", "('advanced', 0.058842976)", "('improve', 0.058842976)", "('learning', 0.05883686)", "('satisfaction', 0.05882615)", "('customers', 0.05882349)", "('machine', 0.05882349)", "('leverage', 0.05882349)", "('data', 0.05881121)", "('analyze', 0.058810115)", "('use', 0.058796745)", "('ai', 0.058796745)", "('sales', 0.058796745)", "('natural', 0.058796745)", "('processing', 0.058796745)"]

["('sales', 0.06382648

C:\Users\manasa\anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [244]:
topic_w={}
for topic_word in topic_words:
    topic_word=dict(topic_word)
    topic_w.update(topic_word)
topic_words_df=pd.DataFrame(topic_w.items(),columns=['word','probability'])

In [245]:
topic_words_df

,word,probability
0,sales,0.000966
1,customer,0.000965
2,processing,0.000965
3,improve,0.000976
4,leverage,0.000966
5,systems,0.000966
6,data,0.000965
7,machine,0.000972
8,natural,0.000965
9,analytics,0.000965


In [190]:
topic_words_df.loc[:,['probability','word']].max()

probability    0.869864
word           use     
dtype: object

In [198]:
from nltk.tokenize import word_tokenize

dt=data.description.loc[0][:500]
tokens=word_tokenize(dt)

In [230]:
topics = lda_train.show_topics(formatted=True,  num_words=20)

In [243]:
[topics[0][1] for el in lda_train[train_id2word.doc2bow(tokens)]]

['0.490*"systems" + 0.444*"learning" + 0.059*"leverage" + 0.001*"machine" + 0.000*"data" + 0.000*"ai" + 0.000*"customers" + 0.000*"analyze" + 0.000*"use" + 0.000*"advanced" + 0.000*"processing" + 0.000*"analytics" + 0.000*"natural" + 0.000*"customer" + 0.000*"sales" + 0.000*"improve" + 0.000*"satisfaction"',
 '0.490*"systems" + 0.444*"learning" + 0.059*"leverage" + 0.001*"machine" + 0.000*"data" + 0.000*"ai" + 0.000*"customers" + 0.000*"analyze" + 0.000*"use" + 0.000*"advanced" + 0.000*"processing" + 0.000*"analytics" + 0.000*"natural" + 0.000*"customer" + 0.000*"sales" + 0.000*"improve" + 0.000*"satisfaction"',
 '0.490*"systems" + 0.444*"learning" + 0.059*"leverage" + 0.001*"machine" + 0.000*"data" + 0.000*"ai" + 0.000*"customers" + 0.000*"analyze" + 0.000*"use" + 0.000*"advanced" + 0.000*"processing" + 0.000*"analytics" + 0.000*"natural" + 0.000*"customer" + 0.000*"sales" + 0.000*"improve" + 0.000*"satisfaction"']

In [226]:
pd.DataFrame([(el[0][0], el[0][1]) for el in lda_train[train_id2word.doc2bow(tokens)]], columns=['topic #', 'weight'])

,topic #,weight
0,13,0.841667
1,0,[13]
2,0,"[(13, 0.9999998)]"


In [255]:
len(bigram_train),len(train_id2word)

(122, 17)

In [260]:
lda_train.show_topics(formatted=False)

[(69,
  [('improve', 0.058823526),
   ('processing', 0.058823526),
   ('analytics', 0.058823526),
   ('sales', 0.058823526),
   ('ai', 0.058823526),
   ('learning', 0.058823526),
   ('use', 0.058823526),
   ('advanced', 0.058823526),
   ('customers', 0.058823526),
   ('analyze', 0.058823526)]),
 (63,
  [('improve', 0.058823526),
   ('processing', 0.058823526),
   ('analytics', 0.058823526),
   ('sales', 0.058823526),
   ('ai', 0.058823526),
   ('learning', 0.058823526),
   ('use', 0.058823526),
   ('advanced', 0.058823526),
   ('customers', 0.058823526),
   ('analyze', 0.058823526)]),
 (42,
  [('improve', 0.058823526),
   ('processing', 0.058823526),
   ('analytics', 0.058823526),
   ('sales', 0.058823526),
   ('ai', 0.058823526),
   ('learning', 0.058823526),
   ('use', 0.058823526),
   ('advanced', 0.058823526),
   ('customers', 0.058823526),
   ('analyze', 0.058823526)]),
 (91,
  [('improve', 0.058823526),
   ('processing', 0.058823526),
   ('analytics', 0.058823526),
   ('sales', 0

In [258]:
data['topics']=topic_words

ValueError: Length of values does not match length of index

In [257]:
data.head()

,id,name,description,metrics,industries,created_at,updated_at,function,topics
0,493,Retargeting,Retarget customers who have already expressed interest in your products or services,Cash flow\nReturn on marketing investment (ROMI),Broadcasting\nFilm Distribution\nMedia and Entertainment\nEnterprise Software\nFinTech\nBanking\nFinancial Services\nArt\nHome Decor\nFashion\nE-Commerce\nRecruiting\nDelivery\nFood and Beverage\nE-Commerce\nFood Processing\nLifestyle\nDietary Supplements\nE-Commerce\nAI Tools\nSupply Chain Management\nMarketplace\nAnalytics\nB2B\nImage Recognition\nDigital Media\nPhotography\nPredictive Analytics\nSEO\nAdvertising\nDirect Marketing\nEmail Marketing\nE-Commerce\nAnalytics\nEvents\nTicketing\nInternet\nTravel\nFashion\nShopping\nE-Commerce\nLifestyle\nFurniture\nFashion\nE-Commerce\nFashion\nE-Commerce\nFitness\nCooking\nHospitality\nMobile\nPersonalization\nFashion\nE-Commerce\nJewelry\nLifestyle\nCustomer Service\nRetail Technology\nInternet\nE-Commerce\nFashion\nManufacturing\nInternet\nE-Commerce\nRetail\nTelecommunications\nRetail\nMobile Devices\nCustomer Service\nLifestyle\nFashion\nRetail Technology\nSoftware Solutions\nE-Commerce\nRetail\nRetail\nOpen Source\nSoftware Solutions\nHardware\nDIY\n3D Printing\nReal Estate\nClassifieds\nE-Commerce\nNews,2020-02-03T15:08:21.159Z,2020-02-03T15:08:21.159Z,Marketing,sales
1,494,Recommendation engine,"Also called recommendation personalization system or recommendation system, these systems leverage customer data to retain and upgrade customers with personalized recommendations via email, site search or other channels",Cash flow\nReturn on marketing investment (ROMI)\nEngagement & conversion\nB2C sales,Search Engine\nRetail\nShopping\nE-Commerce\nLifestyle\nFashion\nRetail\nFashion\nWomen's\nBeauty\nCosmetics\nRetail\nE-Commerce\nFashion,2020-02-03T15:08:21.212Z,2020-02-03T15:08:21.212Z,Marketing,customer
2,495,Social analytics & automation,"Leverage Natural Language Processing and machine vision to analyze and act upon all content generated by your actual or potential customers on social media, surveys and reviews",Customer service effectiveness\nNet Promoter Score (NPS),Cloud Computing\nSoftware Solutions\nInformation Technology\nWeb Hosting\nIaaS\nManufacturing\nAutomotive\nTransportation\nVenture Capital\nFinance\nAngel Investment\nSocial Media\nMarketing\nAdvertising\nBusiness Information Systems\nBusiness Intelligence\nInternet\nAdvertising\nTest and Measurement\nVending and Concessions,2020-02-03T15:08:21.249Z,2020-02-03T15:08:21.249Z,Marketing,processing
3,496,Product Information Management,Manage and improve all your product information centrally to improve product discoverability and appeal,Return on marketing investment (ROMI),Communications Infrastructure\nHardware\nEnterprise Software\nVenture Capital\nFinance\nAngel Investment\nE-Commerce Platforms\nFashion\nBeauty\nRetail\nE-Commerce\nRetail Technology,2020-02-03T15:08:21.270Z,2020-02-03T15:08:21.270Z,Marketing,improve
4,497,Marketing analytics,"Connect all your marketing data and KPIs automatically. Empower your teams to complete and share their advanced analysis with ease. Act on your data to manage campaigns, trigger alerts and improve your marketing efficiency",Return on marketing investment (ROMI),Software Solutions\nCollaboration\nCRM\nData Storage\nHardware\nVirtualization\nFlash Storage\nEnterprise Software\nMarketing Automation\nAdvertising\nTelecommunications,2020-02-03T15:08:21.291Z,2020-02-03T15:08:21.291Z,Marketing,leverage


In [70]:
topics_list=lda_train.print_topics(20,num_words=15)[:10]

### testing model with new unknown data

In [64]:
def get_bigram(df):
    """
    For the test data we only need the bigram data. This is a requirement due to 
    the shapes Gensim functions expect in the test-vector transformation below.
    With both these in hand, we can make the test corpus.
    """
    df['text'] = strip_newline(df)
    words = list(sent_to_words(df.text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
#     lemma = lemmatization(bigram)
    return bigram

In [94]:
#input text to test
test_data=pd.Series('customer improve data satisfaction customers reduce advanced increase efficiency manage uncover passwords insights marketing analyze measure analytics efforts transfer media motivation needed intelligent teams driving bots track learn provide effectively functioning costs trigger personnel optimize improving unless traffic solutions performance agents 24/7 live language multilayared context mining leads handle suggesting logistics garbage pricing understand time issues products dynamic high self employees vehicles cobots medical video legacy create best save personalize label increasing measurement numerous mature retain companies brand flexible real content personal scale operations business unsupervised machine scan protect called effectiveness need intelligence minimize text quickly vision users mimicking autonomous value prices message leverage train triage operate natural phone tools learning voice-controlled transformations matter thousands weeks topic using services generated sales quality overcharged merchandising processing monitor auto patient method emails processes offline page optimization systems')

In [95]:
#get bigrams from test data
bigram_test=get_bigram(test_data)

In [96]:
#get lda model form traning 
lda_train = gensim.models.ldamulticore.LdaMulticore.load('lda_train.model')

In [97]:
# build bow from bigram text data
test_corpus = [train_id2word.doc2bow(text) for text in bigram_test]

In [98]:
#get text vectors
test_vecs = []
for i in range(len(test_data)-1):
    top_topics = lda_train.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    #topic_vec.extend([test_data.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(test_data.iloc[i].text)]) # length review
    test_vecs.append(topic_vec)

In [99]:
 #predict the target function on test/unknown data
sgd_huber.predict(test_vecs)

array(['Healthtech'], dtype='<U17')